KLUE-TC(YNAT) dataset (2800개)

- Labeling Error - 1000개는 라벨을 임의로 바꿈
→ 올바른 라벨로 재맵핑 필요
- Random Noise - 1600개는 text에 노이즈를 추가. 임의의 char 중 20~80%를 랜덤으로 다른 아스키코드로 대체
→ 높은 노이즈를 가지는 text는 제외, 낮은 샘플은 복구가 필요
→ 사람이 개별 판단하지 말 것. 자동 detection이 가능하게 하여 수정하거나, 삭제하거나 진행
- Normal Dataset - 200개 < 이거 맞음???

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

## 데이터 로드

In [ ]:
df = pd.read_csv('./data/train.csv')

## 유틸 함수들

In [ ]:
#매핑 함수

def mapping_label(df):
    # 딕셔너리로 매핑 정의
    label_map = {
        0:'생활문화',1:'스포츠',2:'정치',3:'사회',4:'IT과학',5:'경제',6:'세계'
    }
    
    # map 함수를 사용하여 변환
    df['target'] = df['target'].map(label_map)
    
    return df

#데이터 프레임 정보 출력
def print_df(df):
    print(df.info())
    print(df.describe())
    print(df.head())
    
def target_plot(df):
    target_counts = df['target'].value_counts().sort_index()
    print(target_counts)

    # 시각화
    plt.figure(figsize=(10, 6))
    sns.barplot(x=target_counts.index, y=target_counts.values, palette='viridis')
    plt.title('Count of Each Target Value')
    plt.xlabel('Target Value')
    plt.ylabel('Count')
    plt.xticks(range(0, 7))  # target의 범위에 맞춰 x축 값 설정
    plt.show()    
    
def print_targets(df):
    # target 값에 따라 데이터프레임 정렬
    df = df.sort_values(by='target')

    # 각 target 값에 대해 샘플 출력
    target_values = df['target'].unique()

    for target in target_values:
        print(f"Target: {target}")
        samples = df[df['target'] == target].sample(n=10, random_state=42)
        display(samples)  # Jupyter Notebook에서 DataFrame을 보기 좋게 출력

## 데이터 전처리

In [ ]:
from filter import *

In [ ]:
# 특수 기호의 패턴 정의
# 이 예시에서는 영숫자 이외의 모든 기호를 특수 기호로 간주합니다.
# special_char_pattern = r'[^가-힣A-Z\u4E00-\u9FFF\s0-9]'
# .숫자% 형식을 제외하고, ㎜도 특수 기호에서 제외
special_char_pattern = r'(?<!\d)\.(?!\d)|(?<!\d)%|[^가-힣A-Z\u4E00-\u9FFF\s0-9\.%㎜㎡]'

# 각 text에 포함된 특수 기호의 개수를 세는 함수 정의
def count_special_characters(text):
    return len(re.findall(special_char_pattern, text))

# 데이터프레임에 새로운 열 추가
df['special_char_count'] = df['text'].apply(count_special_characters)
df['special_char_ratio'] = df['special_char_count'] / df['text'].str.len()

df_sorted = df.sort_values(by='special_char_ratio', ascending=False)

# 결과 출력
display(df_sorted.head(20))

In [ ]:
# special_char_ratio가 0.2 이상인 데이터 필터링
df_high_ratio = df_sorted[df_sorted['special_char_ratio'] >= 0.042]
print(len(df_high_ratio))

# 결과 출력
display(df_high_ratio[['text', 'target', 'special_char_count', 'special_char_ratio']])

df_high_ratio['text'] = df_high_ratio['text'].apply(lambda x: re.sub(r'[^가-힣A-Z\u4E00-\u9FFF\s0-9\.%㎜㎡]', '', x))

display(df_high_ratio)

In [ ]:
# df_high_ratio의 인덱스를 이용하여 제외할 행을 찾음
df_excluded = df[~df.index.isin(df_high_ratio.index)]

print(len(df_excluded))
df_excluded.head(20)

In [ ]:
# 결과를 CSV 파일로 저장
df_high_ratio.to_csv('./data/noise_removed.csv', index=False, encoding='utf-8-sig')
#df_excluded.to_csv('./data/noise_free.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#매핑 함수
def mapping_label(df):
    # 딕셔너리로 매핑 정의
    label_map = {
        0:'생활문화',1:'스포츠',2:'정치',3:'사회',4:'IT과학',5:'경제',6:'세계'
    }
    
    # map 함수를 사용하여 변환
    df['target'] = df['target'].map(label_map)
    
    return df

#데이터 프레임 정보 출력
def print_df(df):
    print(df.info())
    print(df.describe())
    print(df.head())
    
def target_plot(df):
    target_counts = df['target'].value_counts().sort_index()
    #print(target_counts)

    # 시각화
    plt.figure(figsize=(10, 6))
    sns.barplot(x=target_counts.index, y=target_counts.values, palette='viridis')
    plt.title('Count of Each Target Value')
    plt.xlabel('Target Value')
    plt.ylabel('Count')
    plt.xticks(range(0, 7))  # target의 범위에 맞춰 x축 값 설정
    plt.show()    
    
def print_targets(df):
    # target 값에 따라 데이터프레임 정렬
    df = df.sort_values(by='target')

    # 각 target 값에 대해 샘플 출력
    target_values = df['target'].unique()

    for target in target_values:
        print(f"Target: {target}")
        samples = df[df['target'] == target].sample(n=10, random_state=42)
        display(samples)  # Jupyter Notebook에서 DataFrame을 보기 좋게 출력

In [ ]:
df = pd.read_csv('data/clean_ascii+df_eng_ger_combined.csv')
target_plot(df)

df = pd.read_csv('output (7).csv')
target_plot(df)

## 데이터 트리밍

In [ ]:
working_df = pd.read_csv('data/df_eng_ger_combined.csv')

# 각 라벨의 최소 개수 찾기
min_count = working_df['target'].value_counts().min()

# 각 클래스에서 최소 개수만큼 샘플링하여 새로운 데이터프레임 생성
df_balanced = working_df.groupby('target').apply(lambda x: x.sample(min_count)).reset_index(drop=True)

df_balanced.to_csv('output/traimmed.csv', index=False, encoding='utf-8-sig')